# Aranet Data Cleaning and Analysis
The purpose of this notebook is to analysis the basic structure of our loaded aranet4 data to check or missing or duplicate records that could cause discontinuities in our time-series forecasting. We will correct and fill/impute missing values accordingly, and write functions to detect and correct for similar errors for data collected in the future.

## Load Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime

# Load the data
# Load the data from the CSV file
aranet4 = pd.read_csv('../datasets/aranet4.csv')
aranetExp = pd.read_csv('../datasets/aranetExp.csv')

# Convert the date column to datetime
aranetExp['date'] = pd.to_datetime(aranetExp['date'])
aranetExp = aranetExp.rename(columns={'date': 'Datetime'})
aranetExp = aranetExp.drop(columns=['id','time'])
aranetExp = aranetExp.set_index('Datetime')

# Convert the date column to datetime 
aranet4['Datetime'] = pd.to_datetime(aranet4['Datetime'])
aranet4 = aranet4.set_index('Datetime')

In [4]:
aranet4.info(), aranetExp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 129396 entries, 2024-01-12 16:08:15 to 2024-04-11 13:43:54
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Carbon dioxide(ppm)        129396 non-null  int64  
 1   Temperature(°F)            129396 non-null  float64
 2   Relative humidity(%)       129343 non-null  float64
 3   Atmospheric pressure(hPa)  126874 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 9.0 MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 240 entries, 2024-03-13 13:04:41 to 2024-04-10 13:32:19
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   door1          240 non-null    object
 1   door2          240 non-null    object
 2   hvac           240 non-null    object
 3   subject_count  240 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ KB


(None, None)

We see we have some `null` values in humidity and pressure, we won't need those values for our current experiment, so we can subset for after `2024-03-13 00:00:00`.

In [3]:
# Filter the DataFrame to include all records from "2024-03-14 00:00:00" onwards
aranet4_sub = aranet4.loc["2024-03-14 00:00:00":].copy()
aranet4_sub.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41144 entries, 2024-03-14 00:00:49 to 2024-04-11 13:43:54
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Carbon dioxide(ppm)        41144 non-null  int64  
 1   Temperature(°F)            41144 non-null  float64
 2   Relative humidity(%)       41144 non-null  float64
 3   Atmospheric pressure(hPa)  41144 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 1.6 MB


In order to join the two data sets we have to put them on a unique common datetime minute index. First, let's round everything to the nearest minute and looking duplicate minutes.

In [5]:
def custom_round_to_nearest_minute(dt_index):
    rounded_index = dt_index.floor('min') + pd.to_timedelta((dt_index.second >= 30).astype(int), unit='min')
    rounded_index.name = dt_index.name  # Preserve the name of the index
    return rounded_index

aranet4_sub.index = custom_round_to_nearest_minute(aranet4_sub.index)
aranetExp.index = custom_round_to_nearest_minute(aranetExp.index)
aranet4_sub.info(), aranetExp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41144 entries, 2024-03-14 00:01:00 to 2024-04-11 13:44:00
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Carbon dioxide(ppm)        41144 non-null  int64  
 1   Temperature(°F)            41144 non-null  float64
 2   Relative humidity(%)       41144 non-null  float64
 3   Atmospheric pressure(hPa)  41144 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 1.6 MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 240 entries, 2024-03-13 13:05:00 to 2024-04-10 13:32:00
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   door1          240 non-null    object
 1   door2          240 non-null    object
 2   hvac           240 non-null    object
 3   subject_count  240 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ KB


(None, None)

Check for duplicates and gaps in the index

In [27]:
import pandas as pd

# Find the indices of the rows where the index is duplicated and the previous indices
duplicate_indices = aranet4_sub.index.duplicated(keep='first')
aranet4_sub[duplicate_indices].index

DatetimeIndex(['2024-03-15 08:21:00', '2024-03-17 02:02:00',
               '2024-03-20 08:38:00', '2024-03-20 16:04:00',
               '2024-03-22 08:24:00', '2024-03-24 08:39:00',
               '2024-03-25 20:19:00', '2024-03-27 08:54:00',
               '2024-03-28 17:56:00', '2024-03-31 06:21:00',
               '2024-03-31 21:57:00', '2024-04-01 01:52:00',
               '2024-04-03 04:28:00', '2024-04-07 00:21:00',
               '2024-04-09 22:20:00'],
              dtype='datetime64[ns]', name='Datetime', freq=None)

In [25]:
aranet4_sub.loc['2024-03-15 08:15:00':'2024-03-15 08:30:00']

,Carbon dioxide(ppm),Temperature(°F),Relative humidity(%),Atmospheric pressure(hPa)
Datetime,,,,
2024-03-15 08:15:00,757,69.9,48.0,1015.2
2024-03-15 08:16:00,748,69.8,48.0,1015.2
2024-03-15 08:17:00,755,69.9,48.0,1015.3
2024-03-15 08:18:00,754,69.9,48.0,1015.2
2024-03-15 08:19:00,763,69.9,48.0,1015.2
2024-03-15 08:20:00,749,69.8,48.0,1015.2
2024-03-15 08:21:00,728,69.9,48.0,1015.1
2024-03-15 08:21:00,760,69.9,48.0,1015.2
2024-03-15 08:22:00,767,70.0,48.0,1015.1


In [30]:
import pandas as pd

# Calculate the time difference between consecutive records
time_diff = aranet4_sub.index.to_series().diff()

# Find the indices where the time difference is greater than 1 minute
one_minute_gaps = time_diff > pd.Timedelta(minutes=1)

# Filter the DataFrame to only include the records immediately after the gaps greater than 1 minute
one_minute_gaps_df = aranet4_sub[one_minute_gaps]

one_minute_gaps_df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15 entries, 2024-03-17 00:09:00 to 2024-04-10 22:33:00
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Carbon dioxide(ppm)        15 non-null     int64  
 1   Temperature(°F)            15 non-null     float64
 2   Relative humidity(%)       15 non-null     float64
 3   Atmospheric pressure(hPa)  15 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 600.0 bytes
